In [1]:
import findspark
findspark.init('/usr/local/spark')

In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
sc= SparkContext()
sqlContext = SQLContext(sc)
house_df = sqlContext.read.format('com.databricks.spark.csv').options(header='false', inferschema='true').load('YearPredictionMSD.txt')
house_df.take(1)

[Row(_c0=2001, _c1=49.94357, _c2=21.47114, _c3=73.0775, _c4=8.74861, _c5=-17.40628, _c6=-13.09905, _c7=-25.01202, _c8=-12.23257, _c9=7.83089, _c10=-2.46783, _c11=3.32136, _c12=-2.31521, _c13=10.20556, _c14=611.10913, _c15=951.0896, _c16=698.11428, _c17=408.98485, _c18=383.70912, _c19=326.51512, _c20=238.11327, _c21=251.42414, _c22=187.17351, _c23=100.42652, _c24=179.19498, _c25=-8.41558, _c26=-317.87038, _c27=95.86266, _c28=48.10259, _c29=-95.66303, _c30=-18.06215, _c31=1.96984, _c32=34.42438, _c33=11.7267, _c34=1.3679, _c35=7.79444, _c36=-0.36994, _c37=-133.67852, _c38=-83.26165, _c39=-37.29765, _c40=73.04667, _c41=-37.36684, _c42=-3.13853, _c43=-24.21531, _c44=-13.23066, _c45=15.93809, _c46=-18.60478, _c47=82.15479, _c48=240.5798, _c49=-10.29407, _c50=31.58431, _c51=-25.38187, _c52=-3.90772, _c53=13.29258, _c54=41.5506, _c55=-7.26272, _c56=-21.00863, _c57=105.50848, _c58=64.29856, _c59=26.08481, _c60=-44.5911, _c61=-8.30657, _c62=7.93706, _c63=-10.7366, _c64=-95.44766, _c65=-82.03307

In [3]:
#house_df.cache() #not enough memory storage
house_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: double (nullable = true)
 |-- _c2: double (nullable = true)
 |-- _c3: double (nullable = true)
 |-- _c4: double (nullable = true)
 |-- _c5: double (nullable = true)
 |-- _c6: double (nullable = true)
 |-- _c7: double (nullable = true)
 |-- _c8: double (nullable = true)
 |-- _c9: double (nullable = true)
 |-- _c10: double (nullable = true)
 |-- _c11: double (nullable = true)
 |-- _c12: double (nullable = true)
 |-- _c13: double (nullable = true)
 |-- _c14: double (nullable = true)
 |-- _c15: double (nullable = true)
 |-- _c16: double (nullable = true)
 |-- _c17: double (nullable = true)
 |-- _c18: double (nullable = true)
 |-- _c19: double (nullable = true)
 |-- _c20: double (nullable = true)
 |-- _c21: double (nullable = true)
 |-- _c22: double (nullable = true)
 |-- _c23: double (nullable = true)
 |-- _c24: double (nullable = true)
 |-- _c25: double (nullable = true)
 |-- _c26: double (nullable = true)
 |-- _c27: double (nullable = tr

In [4]:
'''import pandas as pd
house_df.describe().toPandas().transpose()'''

'import pandas as pd\nhouse_df.describe().toPandas().transpose()'

In [5]:
import six
for i in house_df.columns:
    if not( isinstance(house_df.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation to Year Song Was Published for ", i, house_df.stat.corr('_c0',i))

Correlation to Year Song Was Published for  _c0 1.0
Correlation to Year Song Was Published for  _c1 0.22537353164518467
Correlation to Year Song Was Published for  _c2 0.021401940477790267
Correlation to Year Song Was Published for  _c3 -0.1395409003929431
Correlation to Year Song Was Published for  _c4 -0.003285930718020744
Correlation to Year Song Was Published for  _c5 0.01933810175956383
Correlation to Year Song Was Published for  _c6 -0.1874431648512607
Correlation to Year Song Was Published for  _c7 0.11045983325926205
Correlation to Year Song Was Published for  _c8 -0.034976609716785255
Correlation to Year Song Was Published for  _c9 -0.024795785634134346
Correlation to Year Song Was Published for  _c10 0.03317958273460323
Correlation to Year Song Was Published for  _c11 0.03051394371282924
Correlation to Year Song Was Published for  _c12 -0.0971636958428229
Correlation to Year Song Was Published for  _c13 0.023283586485631707
Correlation to Year Song Was Published for  _c14 0.0

The correlation coefficient ranges from –1 to 1. When it is close to 1, it means that there is a strong positive correlation; for example, the median value tends to go up when the number of rooms goes up. When the coefficient is close to –1, it means that there is a strong negative correlation; the median value tends to go down when the percentage of the lower status of the population goes up. Finally, coefficients close to zero mean that there is no linear correlation.
https://towardsdatascience.com/building-a-linear-regression-with-pyspark-and-mllib-d065c3ba246a

In [6]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['_c1','_c3','_c7','_c36','_c40','_c46','_c63','_c67'], outputCol = 'features')
vhouse_df = vectorAssembler.transform(house_df)
vhouse_df = vhouse_df.select(['features', '_c0'])
vhouse_df.show(3)

+--------------------+----+
|            features| _c0|
+--------------------+----+
|[49.94357,73.0775...|2001|
|[48.73215,70.3267...|2001|
|[50.95714,55.8185...|2001|
+--------------------+----+
only showing top 3 rows



USE the first 463,715 examples as the training dataset and the last 51,630 examples as the test dataset.

In [7]:
#vhouse_df.count()
splits = vhouse_df.randomSplit([0.9, 0.1])
train_df = splits[0]
test_df = splits[1]

In [9]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='_c0', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [0.4077051634535665,-0.051387873629669706,0.019033399321225903,-0.0006451080501216022,-0.002849862895023887,0.0,0.0,-0.0020058913457660817]
Intercept: 1981.320686947038


In [10]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 10.382032
r2: 0.097244


In [11]:
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","_c0","features").show(5)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="_c0",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

+------------------+----+--------------------+
|        prediction| _c0|            features|
+------------------+----+--------------------+
| 1982.070576695173|2003|[7.52015,24.49667...|
|1993.0002349104159|1996|[10.93386,-139.35...|
|1988.1749345823484|1981|[13.89575,-9.1877...|
|1982.6210833583946|2002|[14.26674,66.5688...|
|1989.0740975226968|2005|[14.31269,-44.816...|
+------------------+----+--------------------+
only showing top 5 rows

R Squared (R2) on test data = 0.102986


In [12]:
test_result = lr_model.evaluate(test_df)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

Root Mean Squared Error (RMSE) on test data = 10.3884


In [13]:
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()

numIterations: 11
objectiveHistory: [0.500000000003638, 0.48818196467213887, 0.4670785056942257, 0.4639465089954179, 0.46263879373138417, 0.4625173126366664, 0.4625042618445512, 0.462503415163701, 0.4625033868428803, 0.46250338327221496, 0.46250338276296493]
+------------------+
|         residuals|
+------------------+
|21.999109723026095|
|12.685408781973365|
|20.953710814658052|
| 19.17757654100342|
|11.211666007628537|
|21.829530095515565|
|26.026522196688802|
|  4.24170515102378|
|3.9865618295555123|
|18.501324417992237|
|1.9396985402386235|
| 10.58982946590254|
|23.112719858593664|
| 20.09639042795834|
| 6.072749976447085|
|  20.2705404108342|
| 17.04738195204891|
|15.733245493580398|
| 3.046615567197705|
| 8.044601768599023|
+------------------+
only showing top 20 rows



In [14]:
predictions = lr_model.transform(test_df)
predictions.select("prediction","_c0","features").show()

+------------------+----+--------------------+
|        prediction| _c0|            features|
+------------------+----+--------------------+
| 1982.070576695173|2003|[7.52015,24.49667...|
|1993.0002349104159|1996|[10.93386,-139.35...|
|1988.1749345823484|1981|[13.89575,-9.1877...|
|1982.6210833583946|2002|[14.26674,66.5688...|
|1989.0740975226968|2005|[14.31269,-44.816...|
|1991.4503704439514|2004|[14.43301,-32.922...|
|1989.5814197659454|2000|[15.19295,-35.841...|
|  1985.35752707804|2000|[16.61003,32.7588...|
|1993.0759952549238|1994|[16.67855,-79.629...|
|1994.5444392081677|2009|[16.77987,-143.71...|
| 1987.261039615124|2005|[16.911,10.90929,...|
|1985.6700808067997|2002|[17.14097,-42.829...|
|1995.0779896509655|2003|[17.19291,-72.959...|
| 1995.551051690657|2008|[17.566,-151.1249...|
| 1971.265435581001|2002|[17.80924,250.687...|
|1989.6740031268232|1996|[17.86658,-41.341...|
|1988.2278443779112|2005|[17.96247,-35.737...|
|1981.6877182321386|1991|[17.9813,92.08273...|
|1996.9707622

In [15]:
predictions = lr_model.transform(test_df)
predictions.select("prediction","_c0","features").show()

+------------------+----+--------------------+
|        prediction| _c0|            features|
+------------------+----+--------------------+
| 1982.070576695173|2003|[7.52015,24.49667...|
|1993.0002349104159|1996|[10.93386,-139.35...|
|1988.1749345823484|1981|[13.89575,-9.1877...|
|1982.6210833583946|2002|[14.26674,66.5688...|
|1989.0740975226968|2005|[14.31269,-44.816...|
|1991.4503704439514|2004|[14.43301,-32.922...|
|1989.5814197659454|2000|[15.19295,-35.841...|
|  1985.35752707804|2000|[16.61003,32.7588...|
|1993.0759952549238|1994|[16.67855,-79.629...|
|1994.5444392081677|2009|[16.77987,-143.71...|
| 1987.261039615124|2005|[16.911,10.90929,...|
|1985.6700808067997|2002|[17.14097,-42.829...|
|1995.0779896509655|2003|[17.19291,-72.959...|
| 1995.551051690657|2008|[17.566,-151.1249...|
| 1971.265435581001|2002|[17.80924,250.687...|
|1989.6740031268232|1996|[17.86658,-41.341...|
|1988.2278443779112|2005|[17.96247,-35.737...|
|1981.6877182321386|1991|[17.9813,92.08273...|
|1996.9707622

DECISION TREE REGRESSOR

In [16]:
from pyspark.ml.regression import DecisionTreeRegressor
dt = DecisionTreeRegressor(featuresCol ='features', labelCol = '_c0')
dt_model = dt.fit(train_df)
dt_predictions = dt_model.transform(test_df)
dt_evaluator = RegressionEvaluator(
    labelCol="_c0", predictionCol="prediction", metricName="rmse")
rmse = dt_evaluator.evaluate(dt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 10.1731


In [17]:
dt_model.featureImportances

SparseVector(8, {0: 0.4905, 1: 0.3441, 2: 0.0114, 3: 0.0853, 4: 0.0374, 5: 0.0035, 6: 0.026, 7: 0.0017})

Gradient-boosted tree regression

In [19]:
from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(featuresCol = 'features', labelCol = '_c0', maxIter=10)
gbt_model = gbt.fit(train_df)
gbt_predictions = gbt_model.transform(test_df)
gbt_predictions.select('prediction', '_c0', 'features').show(5)

+------------------+----+--------------------+
|        prediction| _c0|            features|
+------------------+----+--------------------+
|1990.2933017856801|2003|[7.52015,24.49667...|
|1996.6330638881939|1996|[10.93386,-139.35...|
|1996.8264666500281|1981|[13.89575,-9.1877...|
|1993.1170415043696|2002|[14.26674,66.5688...|
|1995.8186633244256|2005|[14.31269,-44.816...|
+------------------+----+--------------------+
only showing top 5 rows



In [20]:
gbt_evaluator = RegressionEvaluator(
    labelCol="_c0", predictionCol="prediction", metricName="rmse")
rmse = gbt_evaluator.evaluate(gbt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 10.0314


In [ ]:
#credits: https://towardsdatascience.com/building-a-linear-regression-with-pyspark-and-mllib-d065c3ba246a